In [1]:
import csv
import pandas
import os
import re
import time
import datetime

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
from openpyxl import load_workbook

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

In [2]:
db_twitter = client["Twitter"]
collections_twitter = db_twitter.collection_names()

In [3]:
# get current year
current_timestamp = int(time.time() * 1000)
current_year = int(datetime.datetime.now().year)
print("current year : " + str(current_year))

# get current week
current_week = int((current_timestamp - 1546214400000)/1000/604800)+1
print("current week : " + str(current_week))

current year : 2019
current week : 11


In [4]:
dic_collection = {}
for i in collections_twitter:
    if i.startswith("20"):
        year = i[0:4]
        week = re.search('_(.+?)_', i).group(1)[1:]
        if int(year) < current_year:
            dic_collection[i] = "{:,}".format(db_twitter[i].find({}).count())
        else:
            try:
                if int(week) < current_week:
                    dic_collection[i] = "{:,}".format(db_twitter[i].find({}).count())
            except: pass

for key in sorted(dic_collection):
    print("%s: %s" % (key, dic_collection[key]))

2018_W51_Twitter_Australia: 20,471
2018_W51_Twitter_Other: 8,978,243
2018_W52_Twitter_Australia: 38,065
2018_W52_Twitter_Other: 16,940,708
2019_W10_Twitter_Australia: 40,767
2019_W10_Twitter_Other: 15,941,350
2019_W1_Twitter_Australia: 40,880
2019_W1_Twitter_Other: 16,994,959
2019_W2_Twitter_Australia: 37,645
2019_W2_Twitter_Other: 14,474,627
2019_W3_Twitter_Australia: 52,348
2019_W3_Twitter_Other: 20,416,992
2019_W4_Twitter_Australia: 59,625
2019_W4_Twitter_Other: 23,417,927
2019_W5_Twitter_Australia: 61,617
2019_W5_Twitter_Other: 23,917,345
2019_W6_Twitter_Australia: 61,435
2019_W6_Twitter_Other: 24,176,122
2019_W7_Twitter_Australia: 28,017
2019_W7_Twitter_Other: 10,484,648
2019_W8_Twitter_Australia: 60,035
2019_W8_Twitter_Other: 23,446,630
2019_W9_Twitter_Australia: 58,316
2019_W9_Twitter_Other: 22,293,795


In [5]:
def get_list_from_csv(data):
    ci = data.city.tolist()
    city = ci[1:]
    st = data.state.tolist()
    state = st[1:]
    co = data.country.tolist()
    country = co[1:]
    return city,state,country

# Extract a hashtag_city report from individual collections

In [ ]:
# read city, state, country from csv file
colnames = ['city', 'state', 'country']
au_data = pandas.read_csv('supporting-files/au.csv', names=colnames, encoding="ISO-8859-1")
world_data = pandas.read_csv('supporting-files/world-cities.csv', names=colnames, encoding="ISO-8859-1")

In [ ]:
pipeline = [
    {"$match": { "entities.hashtags": {"$exists":True,"$ne":[]}}},
    {"$match": { "lang" : "en"}},
    { "$group": {
        "_id": {
            "hashtags": "$entities.hashtags",
            "location": "$user.location"},
        "count": { "$sum": 1 },
        }
    }
]

In [ ]:
while True:
    try:
        csv_columns = ['hashtag','city','state','country','count']

        for collection in sorted(dic_collection):
            if "Australia" in collection:     
                city, state, country = get_list_from_csv(au_data)
            elif "Other" in collection:
                city, state, country = get_list_from_csv(world_data)

            print("-------------------\n")
            print("Processing on collection: " + collection)

            data_format = []
            data_list = list(db_twitter[collection].aggregate(pipeline,allowDiskUse=True))
            c = 0 

            if len(data_list) > 0 : 
                for data in data_list:
                    for htag in data["_id"]["hashtags"]:
                        # get hashtag
                        text = htag["text"].lower()
                        # check if it is in English
                        if(re.match("^[a-zA-Z0-9]*$",text)):
                            exist = 0
                            # check if the location is null
                            if data["_id"]["location"] is not None: 
                                for x in city: 
                                    if isinstance(x,str):
                                        loc = data["_id"]["location"].lower().replace("australia","")
                                        if x.lower() in loc:
                                            index = city.index(x)
                                            if len(data_format)>0:
                                                for d in data_format:
                                                    if(x in d["id"]) and (text in d["hashtag"]):
                                                        d["count"] += data["count"]
                                                        exist = 1
                                                        c+=1
                                                        break 
                                                if exist == 0:
                                                    data_format.append({"id":x + ',' + str(state[index]) + "," + str(country[index]),"hashtag":text,"count":data["count"]})
                                                    exist = 1
                                                    c+=1
                                            else: 
                                                data_format.append({"id":x + ',' + str(state[index]) + "," + str(country[index]),"hashtag":text,"count":data["count"]})
                                                exist = 1
                                                c+=1
                                            print("No." + str(c) + " city: " + x)
                                            break
                                if exist == 0:
                                    for z in state:
                                        if isinstance(z, str):
                                            if z.lower() in data["_id"]["location"].lower():
                                                index = state.index(z)
                                                if len(data_format)>0:
                                                    for d in data_format:
                                                        s = d["id"].split(",")[0]
                                                        if (s == "null") and (z in d["id"]) and (text in d["hashtag"]):
                                                            d["count"] += data["count"]  
                                                            exist = 1
                                                            c+=1
                                                            break 
                                                    if exist == 0:
                                                        data_format.append({"id":"null" + "," + z + "," + str(country[index]),"hashtag":text,"count":data["count"]})
                                                        exist = 1
                                                        c+=1
                                                else: 
                                                    data_format.append({"id": "null" + ","  + z + "," + str(country[index]),"hashtag":text,"count":data["count"]})  
                                                    exist = 1
                                                    c+=1
                                                print("No." + str(c) + " state: " + z)
                                                break
                                if exist == 0:
                                    for y in country:
                                        if isinstance(y,str):
                                            if y.lower() in data["_id"]["location"].lower():
                                                if len(data_format)>0:
                                                    for d in data_format:
                                                        s = d["id"].split(",")[1]
                                                        if (s == "null") and (y in d["id"]) and (text in d["hashtag"]):
                                                            d["count"] += data["count"]       
                                                            exist = 1
                                                            c+=1
                                                            break 
                                                    if exist == 0:
                                                        data_format.append({"id":"null" + "," + "null" + ',' + y,"hashtag":text,"count":data["count"]})
                                                        exist = 1
                                                        c+=1
                                                else: 
                                                    data_format.append({"id": "null" + ","  + "null" + ',' + y,"hashtag":text,"count":data["count"]}) 
                                                    exist = 1
                                                    c+=1
                                                print("No." + str(c) +" country: " + y)
                                                break


                print("hashtag list is finished")

                #create folder if not exist
                folder = "output/hashtag_city/"
                if not os.path.exists(folder):
                    os.makedirs(folder)

                file_name = "output/hashtag_city/{" + collection + "}_hashtag_city.csv"
                with open(file_name, 'w') as f:
                    # header
                    f.write('hashtag,city,state,country,number of tweets\n')

                    for data in data_format:
                        city = data['id'].split(",")[0]
                        state = data['id'].split(",")[1]
                        country = data['id'].split(",")[2]
                        if "Australia" in collection:
                            if state == 'Victoria':
                                state = 'VIC'

                        line = data["hashtag"] + ',' + city + ',' + state + ',' +  country + ',' + str(data['count']) + '\n'
                        f.write(line)

                print ("csv file for collection " + collection + ' is finished.')
                print("-----------------------\n")
    except ValueError:
        print("Oops!  Try again...")


